#### Programmatic or Conditional Version Counters

When *version_id_generator* is set to `False`, we can also __programmatically (and conditionally)__ set the `version identifier` on our object in the same way we assign any other mapped attribute. Such as if we used our _UUID_ example, but set *version_id_generator* to `False`, we can set the `version identifier` at our choosing.

In [ ]:
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

import uuid

In [ ]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
Base = declarative_base()
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [ ]:
class User(Base):
    __tablename__ = "user"
    
    id = Column(Integer, primary_key=True)
    version_uuid = Column(String(32), nullable=False)
    name = Column(String(50), nullable=False)
    
    __mapper_args__ = {"version_id_col": version_uuid, "version_id_generator": False}

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
u1 = User(name="u1", version_uuid=uuid.uuid4())
session.add(u1)
session.commit()

In [ ]:
u1.name = "u2"
u1.version_uuid = uuid.uuid4()
session.commit()

We can _update_ our `User` object __without incrementing the version counter__ as well; the value of the _counter_ will __remain unchanged__, and the `UPDATE` statement will still _check against the previous value_. This may be __useful for schemes__ where only _certain classes of UPDATE_ are __sensitive__ to `concurrency issues`.

In [ ]:
# will leave version_uuid unchanged
u1.name = "u3"
session.commit()